#### Этап 1. Исследовательский анализ (EDA)

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image

Загрузка датасета ингридиентов

In [ ]:
ingredients_df = pd.read_csv('data/ingredients.csv')
ingredients_df.head(3)

In [ ]:
print('Количество ингридиентов:', ingredients_df.shape[0])

In [ ]:
ingredients_df.info()

In [ ]:
ingredients_df.value_counts('ingr')

Стоит удалить deprecated ингредиенты и блюда, в которых они используются?

In [ ]:
dep_ingr_ids = ingredients_df.loc[ingredients_df['ingr'] == 'deprecated', 'id'].to_list()

In [ ]:
ingredients_df = ingredients_df[~ingredients_df['id'].isin(dep_ingr_ids)].reset_index(drop=True)

In [ ]:
print('Количество ингридиентов:', ingredients_df.shape[0])

In [ ]:
ingredients_df.to_csv('data/ingredients_fixed.csv')

Загрузка датасета блюд

In [ ]:
dishes_df = pd.read_csv('data/dish.csv')
dishes_df.head(3)

In [ ]:
print('Количество блюд:', dishes_df.shape[0])

In [ ]:
dep_dish_ids = []
for i, row in dishes_df.iterrows():
    ingredients = [int(item.split('_')[1]) for item in row['ingredients'].split(';')]
    for ingr in dep_ingr_ids:
        if ingr in ingredients:
            dep_dish_ids.append(row['dish_id'])
            break

In [ ]:
dishes_df = dishes_df[~dishes_df['dish_id'].isin(dep_dish_ids)].reset_index(drop=True)

In [ ]:
print('Количество блюд:', dishes_df.shape[0])

In [ ]:
dishes_df.to_csv('data/dish_fixed.csv')

In [ ]:
dishes_df.info()

In [ ]:
some_dishes_df = dishes_df.sample(n=5)

In [ ]:
for i, row in some_dishes_df.iterrows():
    dish_id = row['dish_id']
    calories = row['total_calories']
    print('Калорийность блюда:', calories)
    mass = row['total_mass']
    print('Масса блюда:', mass)
    ingredients = row['ingredients'].split(';')
    ingredients_ids = [int(item.split('_')[1]) for item in ingredients]
    print('Список игредиентов:')
    print(ingredients_df.loc[ingredients_df['id'].isin(ingredients_ids)]['ingr'].to_list())
    
    img = Image.open(os.path.join('data/images', dish_id, 'rgb.png'))
    print(img.size)
    
    plt.figure(figsize=(20, 3))
    plt.imshow(np.array(img))
    plt.show()
    print('-' * 3)

1. Как вы будете решать задачу
- буду использовать мультимодальную модель, которая будет работать с изображениями и табличными данными.

2. Какие подходы к обогащению/аугментации данных примените и почему
- масштабирование;
- геометрические преобразования;
- цветовые преобразования;
- добавление шума.
3. На какие метрики будете ориентироваться при обучении
- MAE;
- loss.

#### Этап 2. Подготовка к обучению

In [1]:
from clearml import Logger, Task
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
import torchmetrics

from src.config import Config
from src.dataset import MultimodalDataset, get_transforms, collate_fn
from src.model import MultimodalModel, set_requires_grad
from src.train import train, validate

/home/ubuntu/dishes_calorie_content/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
Task.set_offline(offline_mode=False)

In [ ]:
task = Task.init(project_name='dishes_calorie_conten', task_name='training-2', auto_connect_frameworks=False)
task.connect(Config, name='Config')

In [ ]:
Config.check_save_path()

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
print('Используемое устрйоство:', device)

Используемое устрйоство: cuda


#### Этап 3. Обучение модели

In [ ]:
train_losses, val_losses, train_metrics, val_metrics = train(Config, device)

In [ ]:
plt.figure(figsize=(5, 5))
plt.title('Loss')
plt.plot(train_losses, label='train')
plt.plot(val_losses, label='val')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(5, 5))
plt.title('Metric')
plt.plot(train_metrics, label='train')
plt.plot(val_metrics, label='val')
plt.legend()
plt.show()

In [ ]:
task.close()

#### Этап 4. Валидация качества

In [4]:
model = MultimodalModel(config=Config).to(device)
set_requires_grad(model.image_model, unfreeze_pattern=Config.IMAGE_MODEL_UNFREEZE)

In [5]:
state_dict = torch.load('models/epoch_80.pth')

In [7]:
model.load_state_dict(state_dict)
model.to(device)
model.eval()

MultimodalModel(
  (mlp): Sequential(
    (0): Linear(in_features=551, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=1024, out_features=512, bias=True)
  )
  (image_model): EfficientNet(
    (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNormAct2d(
            32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (aa): Identity()
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(

In [10]:
val_dataset = MultimodalDataset(config=Config,
                                transforms=get_transforms(config=Config, ds_type='test'),
                                ds_type='test')

val_loader = DataLoader(dataset=val_dataset, batch_size=Config.BATCH_SIZE,
                        shuffle=False, num_workers=Config.NUM_WORKERS,
                        collate_fn=collate_fn)

mae_metric_val = torchmetrics.MeanAbsoluteError().to(device)

for batch in val_loader:
    inputs = {
            'image': batch['image'].to(device),
            'table_data': batch['table_data'].to(device),
            'total_mass': batch['total_mass'].to(device)
        }
    labels = batch['label'].to(device)

    outputs = model(**inputs)
    _ = mae_metric_val(preds=outputs, target=labels)
    
val_mae = mae_metric_val.compute().cpu().numpy()
print(val_mae)
mae_metric_val.reset()

41.604694


41.604694
